I will want to beat the baseline score of $R^2=0.24$ and $RMSE=1,881$.

In [26]:
from __future__ import division
import pandas as pd

In [27]:
import warnings
import seaborn as sns

warnings.filterwarnings("ignore", category=DeprecationWarning)
sns.set_style("whitegrid")
sns.set_context("poster")

from pylab import rcParams
rcParams['figure.figsize'] = 20, 5

import matplotlib.pyplot as plt

%matplotlib inline

In [28]:
from zipcode_mapping import zipcode_mapping

In [29]:
from scipy.sparse import csc_matrix, csr_matrix

In [30]:
df = pd.read_pickle('../data/data_w_transformed_census_and_removed_invalid_rows_and_cols_and_fixed_zips_and_descs_and_dropped_latlongs.pkl')
df.shape

(744372, 37)

In [8]:
df.head(1).T

,905223
CASE_ENQUIRY_ID,101001983804
OPEN_DT,2017-01-08 12:00:16
TARGET_DT,2017-01-10 08:30:00
CLOSED_DT,NaT
CASE_TITLE,Parking Enforcement
SUBJECT,Transportation - Traffic Division
REASON,Enforcement & Abandoned Vehicles
TYPE,Parking Enforcement
Department,BTDT
SubmittedPhoto,False


In [31]:
df.Property_Type = df.Property_Type.fillna('other')

In [32]:
df[['Property_Type', 'neighborhood_from_zip']].isnull().sum()

Property_Type            0
neighborhood_from_zip    0
dtype: int64

## Let's do other features in the original dataset first

In [37]:
old_df = df.copy()

In [39]:
df.head(1)#.loc[:, 'SubmittedPhoto':]

,CASE_ENQUIRY_ID,OPEN_DT,TARGET_DT,CLOSED_DT,CASE_TITLE,SUBJECT,REASON,TYPE,Department,SubmittedPhoto,...,COMPLETION_TIME,school,housing,bedroom,value,rent,income,is_issue_unresolved,zipcode,neighborhood_from_zip
905223,101001983804,2017-01-08 12:00:16,2017-01-10 08:30:00,NaT,Parking Enforcement,Transportation - Traffic Division,Enforcement & Abandoned Vehicles,Parking Enforcement,BTDT,False,...,NaN,20_bachelors,own,3,350000.0,2250,87500,True,2131.0,Roslindale


In [33]:
df = df[['TYPE', 'REASON', 'Department', 'SubmittedPhoto', 'Property_Type', 'Source', 'neighborhood_from_zip', 'zipcode', 'COMPLETION_TIME']]

In [9]:
df.isnull().sum()

TYPE                         0
REASON                       0
Department                   0
SubmittedPhoto               0
Property_Type                0
Source                       0
neighborhood_from_zip        0
zipcode                      0
COMPLETION_TIME          65765
dtype: int64

In [34]:
df = df.dropna()

In [11]:
df.isnull().sum()

TYPE                     0
REASON                   0
Department               0
SubmittedPhoto           0
Property_Type            0
Source                   0
neighborhood_from_zip    0
zipcode                  0
COMPLETION_TIME          0
dtype: int64

In [12]:
df.shape

(678607, 9)

## Dummifying

In [36]:
def dummify(df, column):
    # from Darren's linear regression slides
    print '{} is your baseline'.format(sorted(df[column].unique())[-1])
    dummy = pd.get_dummies(df[column]).rename(columns=lambda x: column+'_'+str(x)).iloc[:,0:len(df[column].unique())-1]
    df = df.drop(column,axis=1) #Why not inplace? because if we do inplace, it will affect the df directly
    return pd.concat([df,dummy],axis=1)

In [37]:
df1 = dummify(df, 'TYPE')
df2 = dummify(df1, 'REASON')
df1 = None
df3 = dummify(df2, 'Department')
df2 = None
df4 = dummify(df3, 'Property_Type')
df3 = None
df5 = dummify(df4, 'Source')
df4 = None
df6 = dummify(df5, 'neighborhood_from_zip')
df6 = None
df7 = dummify(df6, 'zipcode')

Zoning is your baseline
Weights and Measures is your baseline
PWDx is your baseline
other is your baseline
Twitter is your baseline
West Roxbury is your baseline


TypeError: 'NoneType' object has no attribute '__getitem__'

In [ ]:
df_transformed = dummify(df6, 'zipcode')

In [15]:
df7.shape

(678607, 344)

In [94]:
df7.head()

,SubmittedPhoto,COMPLETION_TIME,TYPE_ADA,TYPE_Abandoned Bicycle,TYPE_Abandoned Building,TYPE_Abandoned Vehicles,TYPE_Alert Boston,TYPE_Animal Found,TYPE_Animal Generic Request,TYPE_Animal Lost,...,zipcode_2136.0,zipcode_2146.0,zipcode_2163.0,zipcode_2167.0,zipcode_2199.0,zipcode_2203.0,zipcode_2210.0,zipcode_2215.0,zipcode_2222.0,zipcode_2446.0
905425,True,0.918333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
905235,False,2.325000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
905379,False,2.573333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
905264,False,1.030000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
905447,False,1.681944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Let's run the model

In [13]:
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.cross_validation import ShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Splitting train/test 80/20:

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    df7.drop('COMPLETION_TIME', axis=1), 
    df7.COMPLETION_TIME, 
    test_size=0.2, 
    random_state=300
)

In [15]:
# pipe = make_pipeline(StandardScaler(with_mean=False), LinearRegression())
# pipe = make_pipeline(StandardScaler(with_mean=False), LassoCV(alphas=10e4))
pipe = make_pipeline(LassoCV(verbose=True))

In [16]:
cv = ShuffleSplit(X_train.shape[0], n_iter=1, test_size=0.2, random_state=300)

In [17]:
aa = csr_matrix(X_train.astype(int))

In [24]:
ab = aa.tocsc()

In [19]:
type(aa)

scipy.sparse.csr.csr_matrix

In [18]:
aa.data.nbytes

27692080

In [30]:
ab.data.nbytes

27692080

In [21]:
model = LassoCV(
    cv=cv,
    verbose=100,
    n_jobs=-1
)

In [22]:
model.fit(aa, y_train)

....................................................................................................[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 17.3min finished


KeyboardInterrupt: 

In [21]:
params = {}
model2 = GridSearchCV(pipe, param_grid=params, n_jobs=-1, cv=cv, verbose=10)
model2.fit(aa, y_train);

Fitting 1 folds for each of 1 candidates, totalling 1 fits
[CV]  ................................................................


Process PoolWorker-2:
Process PoolWorker-3:
Process PoolWorker-5:
Process PoolWorker-6:
Process PoolWorker-7:
Process PoolWorker-4:
Process PoolWorker-8:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ubuntu/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/ubuntu/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/home/ubuntu/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ubuntu/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    self.run()
  File "/home/ubuntu/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
    self.r

KeyboardInterrupt: 

Are we in a high bias or high variance situation?

In [51]:
pd.Series(model.best_estimator_.steps[-1][-1].coef_).describe()

count     343.000000
mean      124.410932
std       758.276150
min     -2172.507597
25%        -0.000000
50%        -0.000000
75%         2.745331
max      7404.502586
dtype: float64

In [15]:
pd.Series(model.best_estimator_.steps[-1][-1].coef_).describe()

count     343.000000
mean      124.410932
std       758.276150
min     -2172.507597
25%        -0.000000
50%        -0.000000
75%         2.745331
max      7404.502586
dtype: float64

In [93]:
pd.Series(model.best_estimator_.steps[1][-1].coef_).describe()

count    3.430000e+02
mean     7.650652e+11
std      4.269167e+12
min     -2.961399e+13
25%     -6.690403e+10
50%      1.293448e+09
75%      3.208885e+11
max      3.000261e+13
dtype: float64

In [16]:
pd.DataFrame(model.cv_results_).T

,0
mean_fit_time,252.437
mean_score_time,0.482672
mean_test_score,0.268579
mean_train_score,0.282722
params,{}
rank_test_score,1
split0_test_score,0.268579
split0_train_score,0.282722
std_fit_time,0
std_score_time,0


In [69]:
model.score(X_train, y_train)

0.28085030200379324

In [ ]:
model.score(X_test, y_test)